# Introduction to Automatic Differentiation (AutoDiff)

In this lab, we're going to be doing a deep dive into how libraries like Tensorflow actually work. We'll be discussing and exploring the concept of automatic differentiation, and how we can apply automatic differentiation to arbitary graphs of computation to calculate the gradient with respect to any intermediate point.

Understanding how Autodiff works is not only really cool, but it gives you insight into how complex deep learning libraries work under the hood; this gives you the unique ability to identify any limitations in such libraries, and build tools that either complement or extend on such libraries.

## AutoDiff in a Nutshell

The key concept that backs automatic differentiation is the chain rule from calculus. The chain rule lets us compute the derivative of nested/composed functions by calculating the derivative of each of the constitutent parts.

Specifically, consider the function $f(x, y) = cos(x) * sin(y) + \dfrac{x}{y}$. The corresponding computation graph looks something like this:

![alt text](autodiff.png "Autodifferentiation")

Now, there are two types of automatic differentiation &mdash; **forward mode autodifferentiation**, and **reverse mode autodifferentiation**. While they both compute the same thing, they are slightly different algorithms, with reverse mode autodiff being more efficient, but a little less intuitive.

We'll walk through both types of autodiff, with explanations and practical examples.

### Forward Mode AutoDiff

Just like the chain rule, autodifferentiation works best as a recursive algorithm. Forward Mode Autodifferentiation flows bottom up (starting at the base of the computation graph, flowing upwards), and recursively computes the derivative of each node in terms of its parents. 

Let's define the [partial derivative](https://www.khanacademy.org/math/multivariable-calculus/multivariable-derivatives/partial-derivative-and-gradient-articles/a/introduction-to-partial-derivatives) $\frac{df}{dx}$ as $D$. Then, in Forward Mode AutoDiff, we need to calculate the following quantities:

![alt text](forward.png "Forward Mode")

Forward-mode autodiff starts at the bottom, by initializing $Dx$ and $Dy$ to 1, then propagates upward until it reaches the end of the graph. It's called "forward" mode because derivatives flow "forward" along the graph in the same direction that values flow.

The simplest way to implement forward-mode AD is to overload math operators in your language (Python, in our case) to operate on pairs of `(value, derivative)` instead of just operating on values. In the autodiff literature, these tuples are often called **Dual Numbers**. 

Here is an implementation of a Dual Number class in Python.

In [ ]:
# Class representing a pair of (val, derivative)
class DualNum:

    def __init__(self, val, deriv=0):
        self.val = val
        self.deriv = deriv

    '''
    Define methods for converting a regular Python number into a DualNumber, 
    either as a Variable (we calculate derivatives with respect to Variables) 
    or a Constant (we don't calculate derivatives with respect to constants).
    '''
    # A constant; we don't take derivatives with respect to this
    @staticmethod
    def Constant(x):
        return DualNum(x, deriv=0)

    # A variable; we *do* take derivatives with respect to this
    @staticmethod
    def Variable(x):
        return DualNum(x, deriv=1)
    
    '''
    Helper method to convert a number to a DualNum instance when needed
    '''
    # Convert a number to a constant, if needed
    @staticmethod
    def lift_constant(x):
        if isinstance(x, DualNum):
            return x
        else:
            return DualNum(x)

    '''
    Overload (define) standard math operators (add, subtract, etc.) based on derivitive rules
    '''
    # The following operators take either a number or a Constant
    #    for the second argument
    def __add__(self, other):
        other =  DualNum.lift_constant(other)
        return forward_ad_add(self, other)
    
    def __sub__(self, other):
        other =  DualNum.lift_constant(other)
        return forward_ad_sub(self, other)
    
    def __mul__(self, other):
        other =  DualNum.lift_constant(other)
        return forward_ad_mul(self, other)
    
    def __truediv__(self, other):
        other =  DualNum.lift_constant(other)
        return forward_ad_div(self, other)


Below, we implement the subtract and divide methods:

In [ ]:
def forward_ad_sub(x, y):
    return DualNum(
        x.val - y.val,
        x.deriv - y.deriv
    )

def forward_ad_div(x, y):
    return DualNum(
        x.val / y.val,
        (x.deriv * y.val - x.val * y.deriv) / (y.val * y.val)  # The quotient rule for derivatives
    )


### Checkoff #1

Given the above code, implement the add and multiply methods:

In [ ]:
def forward_ad_add(x, y):
    # TODO: Fill in
    pass

def forward_ad_mul(x, y):
    # TODO: Fill in
    pass

Test that your code works with the following:

In [ ]:
def test1(xval):
    x = DualNum.Variable(xval)
    return x + x

assert test1(5).deriv == 2
assert test1(1).deriv == 2

def test2(xval):
    x = DualNum.Variable(xval)
    return (x + 10) * (x + 5)

# D_x[expr] = D_x[x ** 2 + 15 x + 150] = 2 * x + 15
assert test2(5).deriv == 25
assert test2(0).deriv == 15

def test3(xval):
    x = DualNum.Variable(xval)
    return (x + 10) / (x * x)

# D_x[expr] = D_x[(x + 10) / (x ** 2)] = - 1 / x**2 - 20 / x ** 3
assert test3(1).deriv == -21
assert test3(2).deriv == -0.25 - (20 / 8)

def test4(xval):
    x = DualNum.Variable(xval)
    y = x
    for _ in range(50):
        y = y * x
    return y

assert test4(0).deriv == 0
assert test4(1).deriv == 51
assert test4(2).deriv == 51 * 2 ** 50

print('All tests pass!')

### Reverse Mode Autodifferentiation

In our discussion of forward-mode autodiff, we glossed over a very important limitation: it can only evaluate derivatives with respect to one variable at a time! The tests above don't test this condition, but it's not hard to see why it's true: for an intermediate value $w$ computed during the forward pass, the derivative value stored in the dual number represents $\frac{\partial w}{\partial x}$. That is, these are deratives *with respect to a specific variable*. If we want to compute derivatives with respect to $y$ instead of $x$, then we need to re-run the forward pass again. When we're working with deep neural networks, our variables are neural net parameters, and we can often have thousands or even millions of them. Obviously, performing one forward pass for every variable in this case is not going to be practical.

To remedy this problem, we can use reverse mode autodiff. In reverse mode, instead of computing derivatives starting at the leaves of the graph and working up, we instead start at the output node of the graph and work down (or backwards, hence the name 'reverse-mode'). In other words: if the output of the graph is $f$, for an intermediate value $w$ in the graph, the derivative we compute at that node is $\frac{\partial f}{\partial w}$.

Let's say we want to compute the gradient $[ \frac{\partial f}{\partial x}, \frac{\partial f}{\partial y} ]$ - this corresponds to  $[ \frac{\partial w_7}{\partial w_1} , \frac{\partial w_7}{\partial w_2} ]$ above. More concretely, the derivative computation decomposes into the following:

![alt text](autodiff.png "Autodifferentiation")
![alt text](reverse.png "Reverse Mode AutoDiff")

The simplest way to implement reverse mode autodiff is, as with forward mode, to overload math operators. In this case, instead of the overloaded operators returning a `(value, derivative)` tuple, they should return a `(value, inputs, derivative_function)` tuple. In other words, the program builds up a graph of operations as it runs, and where each tuple is a node and the `inputs` pointers imply edges in the graph (i.e. what nodes were inputs to this node). The `derivative_function` is a function that specifies how to compute **the derivative of the `inputs` with the respect to the output** of the operation.

The derivative computation process starts by designating a target value that you want to calculate the gradient of (e.g. $w_7$ in our examples above). We set the derivative of this node equal to 1 (e.g. $\frac{\partial w_7}{\partial w_7} = 1$, and then start propagating this derivative backward along the graph. The imgage below shows an example of this process for the graph we've been using throughout this lab:

<img src="reverse_graph.png" width="800">

Below, we provide a `Node` class that implements this procedure:

In [ ]:
# Reverse mode AD graph node
class Node:

    # derivative_func: takes self as argument, returns the values
    #    of the input adjoints
    def __init__(self, val, input_nodes=[], derivative_func=None):
        self.val = val
        self.deriv = 0
        self.input_nodes = input_nodes
        self.derivative_func = derivative_func
        # Keeps track of how many times nodes are used as inputs
        self.fanout = 0

    # Initiate a backward pass starting from this node
    def backward(self):
        self.deriv = 1
        self.compute_fanout()
        self.backprop()

    # Compute the out degree of each node reachable from self
    def compute_fanout(self):
        self.fanout += 1
        if self.fanout == 1:
            for node in self.input_nodes:
                node.compute_fanout()

    # Propagate backward along the graph
    def backprop(self):
        if self.derivative_func is not None:
            self.fanout -= 1
            # Wait until this node has accumulated adjoint updates from all of its
            #    uses, then propagate back to the inputs
            if self.fanout == 0:
                input_vals = map(lambda node: node.val, self.input_nodes)
                input_adjoints = self.derivative_func(*input_vals)
                for i in range(len(self.input_nodes)):
                    self.input_nodes[i].deriv += input_adjoints[i] * self.deriv
                    self.input_nodes[i].backprop()

    @staticmethod
    def lift(x):
        if isinstance(x, Node):
            return x
        else:
            return Node(x)

    # Math operators. Again, second arg can be number or Node
    def __add__(self, other):
        other = Node.lift(other)
        return reverse_ad_add(self, other)

    def __sub__(self, other):
        other = Node.lift(other)
        return reverse_ad_sub(self, other)

    def __mul__(self, other):
        other = Node.lift(other)
        return reverse_ad_mul(self, other)

    def __truediv__(self, other):
        other = Node.lift(other)
        return reverse_ad_div(self, other) 

Much of this code is structured as in forward mode. The main difference is that we don't compute derivatives when we construct a `Node`; rather, we keep around a `derivative_function` that specifies how to compute the derivative when we eventually do the backward pass.

What's going on with `fanout`? In a general computation graph, a node might have multiple outgoing edges (i.e. it is used as an input by multiple downstream operations). When we do the backward pass, we don't want to backpropagate from such a node until we've accumulated derivatives from all of its downstream nodes. Thus, we keep track of how many such children it has, and we only trigger backpropagation once all of them have made their contribution to the node's derivative.

Below, we implement the reverse mode versions of the subtraction and division operations:

In [ ]:
def reverse_ad_sub(x, y):
    return Node(
        x.val - y.val,
        [x, y],
        lambda x, y: (1, -1)
    )

def reverse_ad_div(x, y):
    return Node(
        x.val / y.val,
        [x, y],
        lambda x, y: ( 1/y, -x/(y*y) )
    )

It may not be obvious how we came up with these expressions, so to make things clearer, we'll walk through the division example in a bit more detail.

Suppose our computation graph's output (that we want to compute the gradient of) is $f$, and the graph contains an operation $w = x \cdot y$ at some point. During the backward pass, we've calculated up to $\frac{\partial f}{\partial w}$, and now we want to backpropagate through the division operation to compute $\frac{\partial f}{\partial x}$ and $\frac{\partial f}{\partial y}$.

We compute $\frac{\partial f}{\partial x}$ as $\frac{\partial f}{\partial w} \cdot \frac{\partial w}{\partial x}$, where the second term is what `reverse_ad_div` needs to calculate. If we expand that using the quotient rule for derivatives, we get $\frac{\partial w}{\partial x} = (\frac{\partial x}{\partial x} \cdot y - x \cdot \frac{\partial y}{\partial x}) / y^2 = (1 \cdot y - 0) / y^2 = 1 / y$.

Similarly, we compute $\frac{\partial w}{\partial y}$ as $\frac{\partial w}{\partial y} = (\frac{\partial x}{\partial y} \cdot y - x \cdot \frac{\partial y}{\partial y}) / y^2 = (0 - x) / y^2 = -x / y^2$.


### Checkoff #2

Given the above code, implement the reverse method versions of the add and multiply methods:

In [ ]:
def reverse_ad_add(x, y):
    # TODO: Fill this in
    pass

def reverse_ad_mul(x, y):
    # TODO: Fill this in
    pass

Test that your code works with the following:

In [ ]:
def test(x_val, y_val):
    x = Node(x_val)
    y = Node(y_val)
    out1 = x * y + x / y
    out1.backward()
    assert x.deriv == (y_val + 1 / y_val)
    assert y.deriv == (x_val - x_val / y_val ** 2)

    x = Node(x_val)
    y = Node(y_val)
    out2 = x * x * y - x * y * y
    out2.backward()
    assert x.deriv == (2 * x_val * y_val - y_val ** 2)
    assert y.deriv == (x_val ** 2 - 2 * x_val * y_val) 

xy_vals = [(1, 1), (0.5, 1.1), (13.4, 0.2)]

for x_val, y_val in xy_vals:
    test(x_val, y_val)
    
print('All tests pass!')

The one caveat to be aware of with reverse-mode: what we gain in speed (by only needing one pass over the graph to compute all the partial derivatives), we pay for in memory (by having to keep around the complete graph of operations, with all their intermediate values, before we can do a backward pass). This need to keep the graph around is one of the reasons why modern deep learning models need GPUs with such large amounts of memory.

## Conclusion

We hope that this has been a straightforward introduction to some of the ideas of automatic differentiation. Note that we can define arbitrarily complex operations in terms of these primitive operations. 

In this lab, we've worked with computation graphs / derivatives of scalar quantities. In Tensorflow, the quantities we work with are higher-order tensors. However, nothing really changes, except that we need to define the matrix equivalents of the above operations. We can also define activations like ReLU and Softmax accordingly.